Wind characteristics are the most important control in aeolian environments. Wind regime and sand transport potential play a crucial rule to determine the evolution and dynamics of aeolian processes and landforms. In this exercise, we will: (1) analyse wind data from a meteorological station and make wind roses; (2) use the Law of the Wall to determine the roughness length; and (3) calculate resultant drift potential, resultant drift direction and wind directional variability using Fryberger method.

## **Wind roses**
Average 6-hourly wind velocities from 1957 to 2003 were acquired from the meteorological station in Dongsheng, Inner Mongolia. Wind directions were recorded in 16 directions. 1 to 16 represents wind from NNE, NE, ENE ... in the clockwise direction. So 22.5 degree intervals. 17 means no wind.

In [ ]:
# install package for windrose
!pip install windrose

In [ ]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import math
from math import pi

In [ ]:
# read the data
df = pd.read_csv("https://raw.githubusercontent.com/Na-Leeds/BSc_Data_to_Insights_Aeolian/refs/heads/main/raw%20data%20wind%20Dongsheng.csv")

In [ ]:
# check the first 10 rows of data
df.head(10)

In [ ]:
# summary statistics
df.describe()

In [ ]:
# check data types
df.dtypes

In [ ]:
# check the size of data
df.shape

In [ ]:
# df.loc[df['direction']>17]

In [ ]:
# delete rows with odd direction
# df = df.drop(df[df['direction']>17].index)

In [ ]:
# check the size of data again
#df.shape

In [ ]:
# check data
df.min()

In [ ]:
df.max()

In [ ]:
# create a new column with direction in degree
df['dir_compass'] = df['direction']*22.5

In [ ]:
# plot histogram of wind speed
df['speed'].hist(bins = 20, rwidth=0.8, color = 'grey')

In [ ]:
# plot wind roses
from windrose import WindroseAxes, plot_windrose

# plot windrose in bar mode, in percent
ax = WindroseAxes.from_ax()

ax.bar(df.dir_compass, df.speed, bins = np.array([0, 2, 4, 6, 8, 10]), normed=True, opening=0.8, cmap=cm.jet, linewidth=0.5, edgecolor='white')
ax.set_legend(title = 'Wind Speed (m/s)', loc='best')

# Format radius axis to percentages
import matplotlib.ticker as mtick
fmt = '%.0f%%'
yticks = mtick.FormatStrFormatter(fmt)
ax.yaxis.set_major_formatter(yticks)

plt.show()

# check this link and choose your favorite colourmap: https://matplotlib.org/stable/gallery/color/colormap_reference.html




In [ ]:
# plot windrose in filled mode, in bin limits
ax = WindroseAxes.from_ax()
ax.contourf(df.dir_compass, df.speed, bins=np.arange(0, 8, 1), cmap=cm.gist_rainbow)
ax.set_legend(title = 'Wind Speed (m/s)', loc='best')

# Format radius axis
fmt = '%.0f'
yticks = mtick.FormatStrFormatter(fmt)
ax.yaxis.set_major_formatter(yticks)

In [ ]:
# plot windrose for each month
def plot_windrose_subplots(data, *, direction, var, color=None, **kwargs):
    """wrapper function to create subplots per axis"""
    ax = plt.gca()
    ax = WindroseAxes.from_ax(ax=ax)
    plot_windrose(direction_or_df=data[direction], var=data[var], ax=ax, **kwargs)


# this creates the raw subplot structure with a subplot per value in month.
g = sns.FacetGrid(
    data=df,
    # the column name for each level a subplot should be created
    col="month",
    # place a maximum of 3 plots per row
    col_wrap=3,
    subplot_kws={"projection": "windrose"},
    sharex=False,
    sharey=False,
    despine=False,
    height=3.5,
)

g.map_dataframe(
    plot_windrose_subplots,
    direction="dir_compass",
    var="speed",
    normed=True,
    # manually set bins, so they match for each subplot
    bins=(0.1, 1, 2, 3, 4, 5, 6, 7),
    calm_limit=0.1,
    kind="bar",
)

# make the subplots easier to compare, by having the same y-axis range
y_ticks = range(0, 20, 4)
for ax in g.axes:
    ax.set_legend(
        title='Wind Speed (m/s)', bbox_to_anchor=(1.15, -0.1), loc="lower right"
    )
    ax.set_rgrids(y_ticks, y_ticks)

# adjust the spacing between the subplots to have sufficient space between plots
plt.subplots_adjust(wspace=0.2)

## **The Law of the Wall - wind profiles**
In order to caculate the threshold velocity at the height of 10.4 m, where cup anemometers were set up at the Dongsheng meteorological station, we will estimate the roughness length of the surface (z0) with wind velocity and direction data collected with a tower of 3-cup anemoanemometers at heights of 0.6m, 1.0m, 1.2m, 1.5m and 2.0m.


In [ ]:
# load the data
df_wind = pd.read_csv("https://raw.githubusercontent.com/Na-Leeds/BSc_Data_to_Insights_Aeolian/refs/heads/main/windprofile.csv")

In [ ]:
# check the data
df_wind.head()

In [ ]:
# There are seven sets of data. We will fit a linear regression between veolicities and ln(H) for each set of data.
from sklearn.metrics import r2_score

df_wind_results = pd.DataFrame(index = range(7), columns=['r_squared', 'shear_v', 'z0'])
plt.figure()
for idata in list(range(0,df_wind.shape[0],1)):

    # perform linear regression
    x_data = df_wind.iloc[:, idata+1]
    y_data = np.log(df_wind['H'])
    slope, intercept = np.polyfit(x_data, y_data, 1)

    # calculate predicted values using the regression equation
    y_predicted = slope * x_data + intercept

    # Calculate R-squared value
    r_squared = r2_score(y_data, y_predicted)
    shear_v = 0.4/slope
    z0 = np.exp(intercept)

    df_wind_results.iloc[idata, 0] = r_squared
    df_wind_results.iloc[idata, 1] = shear_v
    df_wind_results.iloc[idata, 2] = z0


    # Plot the data and regression line
    plt.scatter(x_data, df_wind['H'])
    plt.plot(x_data, np.exp(y_predicted))

# format figure
plt.yscale("log")
plt.xlabel('wind velocity (m/s)')
plt.ylabel('Height (m)')
plt.legend(["data","linear regression"], fontsize="x-large")
plt.show()


In [ ]:
# show results
print(df_wind_results)

In [ ]:
# calculate shear velocity at the height = 10.4m where meteorological station is located. The threshold shear veolicity at the surface is 0.19, which is known.
threshold_shear_v = 0.19
z0_mean = df_wind_results['z0'].mean()

# caluclate threshold shear velocity at the height of 10.4m (height of cup anemometer for the meterological station)
shear_v_station = threshold_shear_v/0.4*math.log(10.4/z0_mean)
print(shear_v_station)

##**Sand drift potential**
Sand drift potentials (DP), which reflect the relative amount of potential sand drift for a certain period of time, were evaluated by the Fryberger method (Fryberger, 1979).

In [ ]:
# calculate drift potential for each direction
df['DP'] = df['speed']*df['speed']*(df['speed']-shear_v_station)*1/df.shape[0]

# change negative DP to zero
df.loc[df['DP']<0, 'DP'] = 0

# check DP
df['DP'].describe()

In [ ]:
# summarise DP for each wind direction
df_DP = pd.DataFrame()
df_DP['dir'] = list(range(1, 17, 1))
df_DP['DP'] = np.nan

for dir in list(range(1, 17, 1)):
    DP_sum = df.loc[df['direction'] == dir, 'DP'].sum()
    df_DP.iloc[dir-1, 1] = DP_sum

print(df_DP)

In [ ]:
# add a column with directions in degree
df_DP['dir_compass'] = df_DP['dir']*22.5
print(df_DP)

In [ ]:
# cacluate resultant drift potential (RDP), remember compass direction is different from normal x,y direction
df_DP['DP_n'] = np.nan
df_DP['DP_e'] = np.nan
for i in list(range(0, 16, 1)):
    df_DP.iloc[i,3] =  df_DP.iloc[i,1] * math.cos(math.radians(df_DP.iloc[i, 2]))
    df_DP.iloc[i,4] =  df_DP.iloc[i,1] * math.sin(math.radians(df_DP.iloc[i, 2]))

RDP_n = df_DP['DP_n'].sum()
RDP_e = df_DP['DP_e'].sum()
RDP = math.sqrt(math.pow(df_DP['DP_n'].sum(), 2) + math.pow(df_DP['DP_e'].sum(), 2))
RDP_dir = math.degrees(math.atan2(RDP_n,RDP_e)) # direction of RDP is direction of wind blows to (not from)
RDP2DP = RDP/df_DP['DP'].sum()

print(RDP_n, RDP_e, RDP, RDP_dir, RDP2DP)

In [ ]:
# Plot DP in polar plot
theta = np.linspace(0, 2*np.pi, 16, endpoint=False)
directions = df_DP['dir_compass']

ax = plt.subplot(polar=True)
ax.bar(theta, df_DP['DP'], width=2*pi/16*0.8, linewidth=1, edgecolor='k', alpha=1, color = 'k')

ax.set_xticks(theta)
ax.set_xticklabels(directions)
ax.set_yticks([2,4,6,8])
ax.set_yticklabels([2,4,6,8])

# set change in clockwise direction and zero at the north direction
ax.set_theta_offset(math.radians(45+22.5))
ax.set_theta_direction(-1)



In [ ]:
# check outputs
df_DP

In [ ]:
# calculate DP for each month
df_DP['DP_Jan'] = np.nan
df_DP['DP_Feb'] = np.nan
df_DP['DP_Mar'] = np.nan
df_DP['DP_Apr'] = np.nan
df_DP['DP_May'] = np.nan
df_DP['DP_Jun'] = np.nan
df_DP['DP_Jul'] = np.nan
df_DP['DP_Aug'] = np.nan
df_DP['DP_Sep'] = np.nan
df_DP['DP_Oct'] = np.nan
df_DP['DP_Nov'] = np.nan
df_DP['DP_Dec'] = np.nan

# create dataframe with 16 rows (wind directions), and 12 coloumns (12 months)
df_DP_n = pd.DataFrame(index = range(16), columns=['Jan', 'Feb', 'Mar', 'Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
df_DP_e = pd.DataFrame(index = range(16), columns=['Jan', 'Feb', 'Mar', 'Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])

# create a dataframe with 12 rows (12 months) and 2 coloumns (resultant drift potential and its direction)
RDP_month = pd.DataFrame(index = range(12), columns=['RDP', 'Dir', 'DP', 'RDP2DP'])

for j in list(range(1,13,1)):
    df_month = df.loc[df['month'] == j]

    for i in list(range(1, 17, 1)):
        DP_sum = df_month.loc[df_month['direction'] == i, 'DP'].sum()
        df_DP.iloc[i-1,j+4] = DP_sum
        df_DP_n.iloc[i-1,j-1] = DP_sum * math.cos(math.radians(df_DP.iloc[i-1, 2]))
        df_DP_e.iloc[i-1,j-1] = DP_sum * math.sin(math.radians(df_DP.iloc[i-1, 2]))

# check results
RDP_month_n = df_DP_n.sum()
RDP_month_e = df_DP_e.sum()
RDP_month_DP = df_DP.sum().iloc[5:17]

for k in list(range(0, 12, 1)):
    RDP = math.sqrt(math.pow(RDP_month_n.iloc[k], 2) + math.pow(RDP_month_e.iloc[k], 2))
    Dir = math.degrees(math.atan2(RDP_month_n.iloc[k], RDP_month_e.iloc[k])) # direction of RDP is direction of wind blows to (not from)
    if Dir < 0:
        Dir = Dir + 180
    RDP_month.iloc[k,0] = RDP
    RDP_month.iloc[k,1] = Dir
    RDP_month.iloc[k,2] = RDP_month_DP.iloc[k]
    RDP_month.iloc[k,3] = RDP/RDP_month_DP.iloc[k]

print(RDP_month)

In [ ]:
# plot DP for each month
theta = np.linspace(0, 2*np.pi, 16, endpoint=False)
directions = df_DP['dir_compass']
fig, axs = plt.subplots(4, 3, subplot_kw = {'projection' : 'polar'}, figsize=(10, 15))
grid = plt.GridSpec(4, 3, wspace=0.2, hspace=1)
plt.figure(figsize=[600,50])
for j in range(4):
    for i in range(3):
        month = j*3 + i + 1
        axs[j,i].bar(theta, df_DP.iloc[: ,5+month-i], width=2*pi/16*0.8, linewidth=1, edgecolor='k', alpha=1, color = 'k')
        axs[j,i].set_xticks(theta)
        axs[j,i].set_xticklabels(directions)
        axs[j,i].set_yticks([0.2,0.4,0.6,0.8,1.0])
        axs[j,i].set_yticklabels([0.2,0.4,0.6,0.8,1.0])

        # set change in clockwise direction and zero at the north direction
        axs[j,i].set_theta_offset(math.radians(45+22.5))
        axs[j,i].set_theta_direction(-1)
        axs[j,i].title.set_text(f'Month {month}')

fig.tight_layout(h_pad=1, w_pad=1)

## **Questions:**
- Comparing the drift potential roses with wind roses in the first part, what do you notice?
- How many dominant wind directions?
- What types of dunes are likely to develop in the given wind regime?